In [1]:
import os 


## Model

In [ ]:
from langchain.chat_models import ChatOpenAI

llm_model = ''

chat = ChatOpenAI(temperature = 0, 
                  model = llm_model)



## Prompt Template

In [ ]:
from langchain.prompts import ChatPromptTemplate

style = 'O mesmo estilo de fala que o batman teria, o tom de batman sério'
template_string = f'''Trauza o texto delimitado por "", para o estilo {style}'''

prompt_template = ChatPromptTemplate.from_template(template_string)

